# María Carolina Navarro Monge
# Caso 2

## Un portafolio de inversión consiste de 1000 acciones de Microsoft. Use el enfoque paramétrico para estimar el riesgo del portafolio a través del VaR y el ES . Compare los resultados cuando se emplean los métodos siguientes para calcular la volatilidad: el estimador clásico insesgado, el estimador EWMA y el estimador GARCH.

In [73]:
# se carga la base de datos

import pandas as pd

Microsoft =  pd.read_csv(r'MSFT.csv')

In [74]:
import numpy as np
#se obtienen los rendimientos 

Rendimientos_M = np.log(Microsoft["Close"] / Microsoft["Close"].shift(1))
Rendimientos_M

0           NaN
1     -0.030018
2     -0.001392
3      0.017381
4     -0.007163
         ...   
122    0.000846
123   -0.007860
124    0.006119
125    0.009684
126   -0.018632
Name: Close, Length: 127, dtype: float64

## Cálculo VaR y ES

Suponiendo que los rendimientos poseen una distribución normal, el VaR y ES se obtienen a partir de las siguientes fórmulas

$$VaR = \mu_L + z_\alpha\cdot\sigma_L$$
$$ES = \mu_L + \sigma\cdot\dfrac{\exp(\dfrac{-z^2_\alpha}{2})}{\sqrt(2\pi)(1-\alpha)}$$


donde $\mu_L = -V_t\cdot\mu_{R_{t+1}}$, $\sigma_L = V_t\cdot\sigma_{R_{t+1}}$, $V_t=N\cdot S_t$, $z_\alpha = \Phi^{-1}(\alpha)$


Se calcularán el VarR y ES al 95%. 

Se obtendrá la media y desviación de los rendimientos empleando una ventana móvil de 30 y 60 para comparar los resultados.

### Caso 1: estimador insesgado usual

$$\mu_R = \dfrac{1}{m}\sum_{i = 1}^m R_{t-i}$$
$$\sigma^2_R = \dfrac{1}{m}\sum_{i = 1}^m (R_{t-i}-\mu_R)^2$$

In [75]:
#se define una ventana móvil para el cálculo de la media y la varianza de los rendimientos
M_media_30_caso1 = Rendimientos_M.rolling(30).mean()
M_media_60_caso1 = Rendimientos_M.rolling(60).mean()
M_media_caso1 = Rendimientos_M.mean()
print(M_media_30_caso1.iloc[-1])
print(M_media_60_caso1.iloc[-1])
print(M_media_caso1)

-0.0026214649303594718
-0.0006028119280967327
-0.0001055189253870471


In [76]:
M_std_30_caso1 = Rendimientos_M.rolling(30).std()
M_std_60_caso1 = Rendimientos_M.rolling(60).std()
M_std_caso1 = Rendimientos_M.std()
print(M_std_30_caso1.iloc[-1])
print(M_std_60_caso1.iloc[-1])
print(M_std_caso1)

0.014068745951918187
0.012618386981224404
0.012838045221790764


In [77]:
from scipy.stats import norm

alpha = 0.95

#cuantil
z = norm.ppf(0.95)
print(z)

1.6448536269514722


In [78]:
# Monto actual de inversión
V_t_M = 1000*Microsoft["Close"].iloc[-1]
print(V_t_M)

409440.002


In [79]:
#Media pérdidas

M_media_perdidas_30_caso1 = -V_t_M*M_std_30_caso1
M_media_perdidas_60_caso1 = -V_t_M*M_std_60_caso1
M_media_perdidas_caso1 = -V_t_M*M_media_caso1
print(M_media_perdidas_30_caso1.iloc[-1])
print(M_media_perdidas_60_caso1.iloc[-1])
print(M_media_perdidas_caso1)

-5760.307370690874
-5166.4723908292935
43.203669021510414


In [80]:
# Desviación pérdidas
M_desviacion_30_caso1 = V_t_M*M_std_30_caso1
M_desviacion_60_caso1 = V_t_M*M_std_60_caso1
M_desviacion_caso1 = V_t_M*M_std_caso1
print(M_desviacion_30_caso1.iloc[-1])
print(M_desviacion_60_caso1.iloc[-1])
print(M_desviacion_caso1)

5760.307370690874
5166.4723908292935
5256.4092612861


In [81]:
#VaR
M_VaR_30_caso1 = M_media_perdidas_30_caso1 + z*M_desviacion_30_caso1
M_VaR_60_caso1 = M_media_perdidas_60_caso1 + z*M_desviacion_60_caso1
M_VaR_caso1 = M_media_perdidas_caso1 +z*M_desviacion_caso1
print(M_VaR_30_caso1.iloc[-1])
print(M_VaR_60_caso1.iloc[-1])
print(M_VaR_caso1)



3714.5551003453093
3331.6184597709134
8689.227507189262


In [82]:
#ES 
M_ES_30_caso1 = M_media_perdidas_30_caso1 + M_std_30_caso1 * (np.exp(-z**2 / 2) / (np.sqrt(2 * np.pi) * (1 - alpha)))
M_ES_60_caso1 = M_media_perdidas_60_caso1 + M_std_60_caso1 * (np.exp(-z**2 / 2) / (np.sqrt(2 * np.pi) * (1 - alpha)))
M_ES_caso1 = M_media_perdidas_caso1 + M_std_caso1 * (np.exp(-z**2 / 2) / (np.sqrt(2 * np.pi) * (1 - alpha)))
print(M_ES_30_caso1.iloc[-1])
print(M_ES_60_caso1.iloc[-1])
print(M_ES_caso1)

-5760.278350908414
-5166.446362720857
43.230150221812764


### Caso 2: Estimador EWMA

$$\sigma^2_R = \dfrac{1-\lambda}{1-\lambda^m}\sum_{i = 1}^m \lambda^{i-1}(R_{t-i}-\mu_R)^2$$

In [83]:
M_media_caso2 = Rendimientos_M.ewm(span=20,adjust=False).mean()
M_std_caso2 = Rendimientos_M.ewm(span=20,adjust=False).std()
print(M_media_caso2.iloc[-1])
print(M_std_caso2.iloc[-1])


-0.0017169462719957432
0.011821083931285555


In [84]:
# Media pérdidas

M_media_perdidas_caso2= -V_t_M*M_media_caso2
print(M_media_perdidas_caso2.iloc[-1])


702.9864850398296


In [85]:
# Desviación pérdidas
M_desviacion_caso2 = V_t_M*M_std_caso2
print(M_desviacion_caso2.iloc[-1])

4840.024628467725


In [86]:
#VaR
M_VaR_caso2 = M_media_perdidas_caso2+ z*M_desviacion_caso2
print(M_VaR_caso2.iloc[-1])

8664.11854970942


In [87]:
#ES 
M_ES_caso2 = M_media_perdidas_caso2 + M_std_caso2 * (np.exp(-z**2 / 2) / (np.sqrt(2 * np.pi) * (1 - alpha)))
print(ES.iloc[-1])

29.050983278026713


### Caso 3: GARCH(1,1)

In [88]:
!pip install arch

from arch import arch_model

In [89]:
# Ajustar el modelo GARCH(1,1)
model = arch_model(Rendimientos_M.dropna(), vol='Garch', p=1, q=1)
garch_fit = model.fit()

# Obtener la media
M_media_caso3 = garch_fit.params['mu']
print(M_media_caso3)

# Obtener la desviación estándar
M_std_caso3 = garch_fit.conditional_volatility
print(M_std_caso3.iloc[-1])

Iteration:      1,   Func. Count:      6,   Neg. LLF: 24533531.4143253
Iteration:      2,   Func. Count:     16,   Neg. LLF: -370.8457761164025
Optimization terminated successfully    (Exit mode 0)
            Current function value: -370.84577620011356
            Iterations: 6
            Function evaluations: 16
            Gradient evaluations: 2
-7.088914745481049e-05
0.012668075873532927


/home/codespace/.python/current/lib/python3.12/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001635. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


In [90]:
# Media pérdidas

M_media_perdidas_caso3= -V_t_M*M_media_caso3
print(M_media_perdidas_caso3)

29.024852675675902


In [91]:
# Desviación pérdidas
M_desviacion_caso3 = V_t_M*M_std_caso3
print(M_desviacion_caso3.iloc[-1])


5186.8170109954735


In [92]:
#VaR
M_VaR_caso3 = M_media_perdidas_caso3 + z*M_desviacion_caso3
print(M_VaR_caso3.iloc[-1])

8560.579625545175


In [93]:
#ES 
M_ES_caso3 = M_media_perdidas_caso3 + M_std_caso3 * (np.exp(-z**2 / 2) / (np.sqrt(2 * np.pi) * (1 - alpha)))
print(M_ES_caso3.iloc[-1])

29.050983278026713


# Repita el ejercicio anterior si, además de las acciones de Microsoft, se tienen 2550 acciones de NVidia.

In [94]:
# se carga la base de datos

NVidia =  pd.read_csv(r'NVDA.csv')

In [58]:
# se obtienen los rendimientos
Rendimientos_N = np.log(NVidia["Close"] / NVidia["Close"].shift(1))
Rendimientos_N

0           NaN
1      0.008493
2      0.031331
3      0.043774
4     -0.057075
         ...   
122    0.014445
123   -0.021189
124   -0.065978
125    0.015024
126   -0.100097
Name: Close, Length: 127, dtype: float64

### Caso 1: Estimador usual

In [95]:
#se define una ventana móvil para el cálculo de la media y la varianza de los rendimientos
N_media_30_caso1 = Rendimientos_N.rolling(30).mean()
N_media_60_caso1 = Rendimientos_N.rolling(60).mean()
N_media_caso1 = Rendimientos_N.mean()
print(N_media_30_caso1.iloc[-1])
print(N_media_60_caso1.iloc[-1])
print(N_media_caso1)

-0.004481125707935915
-0.0018940466279149387
0.0018785366290270163


In [96]:
N_std_30_caso1 = Rendimientos_N.rolling(30).std()
N_std_60_caso1 = Rendimientos_N.rolling(60).std()
N_std_caso1 = Rendimientos_N.std()
print(N_std_30_caso1.iloc[-1])
print(N_std_60_caso1.iloc[-1])
print(N_std_caso1)

0.04951060984489657
0.04190632296081505
0.03744769177985934


In [97]:
# Monto actual de inversión
V_t_N =  2550*NVidia["Close"].iloc[-1]
print(V_t_N)

275400.0


In [127]:
#Media pérdidas

media_perdidas_30_caso1 = -V_t_M*M_media_30_caso1 -V_t_N*N_media_30_caso1
media_perdidas_60_caso1 = -V_t_M*M_media_60_caso1 -V_t_N*N_media_60_caso1
media_perdidas_caso1 = -V_t_M*M_media_caso1 -V_t_N*N_media_caso1
print(media_perdidas_30_caso1.iloc[-1])
print(media_perdidas_60_caso1.iloc[-1])
print(media_perdidas_caso1)

2307.434626294863
768.4357583733241
-474.14531861252993


In [117]:
# correlación
correlacion = np.corrcoef(Rendimientos_M[1:], Rendimientos_N[1:])[0,1]
print(correlacion)

0.4299680447008907


In [124]:
# desviacion perdidas
import math

factor = math.sqrt(M_std_30_caso1.iloc[-1] * N_std_30_caso1.iloc[-1])

# Calcular la desviación estándar de pérdidas
std_perdidas_30_caso_1 = math.sqrt(
    (V_t_M**2) * (M_std_30_caso1.iloc[-1]**2) +
    (V_t_N**2) * (N_std_30_caso1.iloc[-1]**2) +
    2 * correlacion * V_t_M * V_t_N * factor
)

print(std_perdidas_30_caso_1)

factor = math.sqrt(M_std_60_caso1.iloc[-1] * N_std_60_caso1.iloc[-1])

std_perdidas_60_caso_1 = math.sqrt(
    (V_t_M**2) * (M_std_60_caso1.iloc[-1]**2) +
    (V_t_N**2) * (N_std_60_caso1.iloc[-1]**2) +
    2 * correlacion * V_t_M * V_t_N * factor
)

print(std_perdidas_60_caso_1)


factor = math.sqrt(M_std_caso1 * N_std_caso1)

std_perdidas_caso_1 = math.sqrt(
    (V_t_M**2) * (M_std_caso1**2) +
    (V_t_N**2) * (N_std_caso1**2) +
    2 * correlacion * V_t_M * V_t_N * factor
)

print(std_perdidas_caso_1)



52709.19361098396
48884.22531587317
47540.33289554641


In [125]:
#VaR
VaR_30_caso1 = media_perdidas_30_caso1 + z*std_perdidas_30_caso_1
VaR_60_caso1 = media_perdidas_60_caso1 + z*std_perdidas_60_caso_1
VaR_caso1 = media_perdidas_caso1 +z*std_perdidas_caso_1
print(VaR_30_caso1.iloc[-1])
print(VaR_60_caso1.iloc[-1])
print(VaR_caso1)

89006.3429110092
81175.83106990028
77722.74367110737


In [126]:
#ES 
ES_30_caso1 = media_perdidas_30_caso1 + std_perdidas_30_caso_1 * (np.exp(-z**2 / 2) / (np.sqrt(2 * np.pi) * (1 - alpha)))
ES_60_caso1 = media_perdidas_60_caso1 + std_perdidas_60_caso_1 * (np.exp(-z**2 / 2) / (np.sqrt(2 * np.pi) * (1 - alpha)))
ES_caso1 = media_perdidas_caso1 + std_perdidas_caso_1 * (np.exp(-z**2 / 2) / (np.sqrt(2 * np.pi) * (1 - alpha)))
print(ES_30_caso1.iloc[-1])
print(ES_60_caso1.iloc[-1])
print(ES_caso1)

111031.36336106005
101602.55340250365
97587.90821819763


# Caso 2: EWMA

In [128]:
#Media y desviación
N_media_caso2 = Rendimientos_N.ewm(span=20,adjust=False).mean()
N_std_caso2 = Rendimientos_N.ewm(span=20,adjust=False).std()
print(N_media_caso2.iloc[-1])
print(N_std_caso2.iloc[-1])

-0.010503768703750062
0.04847922477394599


In [129]:
#Media pérdidas

media_perdidas_caso2 = -V_t_M*M_media_caso2 -V_t_N*N_media_caso2
print(media_perdidas_caso2.iloc[-1])

3595.724386052597


In [130]:
# Desviación pérdidas
factor = math.sqrt(M_std_caso2.iloc[-1] * N_std_caso2.iloc[-1])

# Calcular la desviación estándar de pérdidas
std_perdidas_caso2 = math.sqrt(
    (V_t_M**2) * (M_std_caso2.iloc[-1]**2) +
    (V_t_N**2) * (N_std_caso2.iloc[-1]**2) +
    2 * correlacion * V_t_M * V_t_N * factor
)

print(std_perdidas_caso2)

50229.03799967118


In [131]:
#VaR
VaR_caso2 = media_perdidas_caso2+ z*std_perdidas_caso2
print(VaR_caso2.iloc[-1])


86215.13971809504


In [139]:
#ES 
ES_caso2 = media_perdidas_caso2 + std_perdidas_caso2 * (np.exp(-z**2 / 2) / (np.sqrt(2 * np.pi) * (1 - alpha)))
print(ES_caso2.iloc[-1])

107203.8043767515


# Caso 3: GARCH(1,1)

In [136]:
# Ajustar el modelo GARCH(1,1)
model_N = arch_model(Rendimientos_N.dropna(), vol='Garch', p=1, q=1)
garch_fit_N = model.fit()

# Obtener la media
N_media_caso3 = garch_fit_N.params['mu']
print(N_media_caso3)

# Obtener la desviación estándar
N_std_caso3 = garch_fit_N.conditional_volatility
print(N_std_caso3.iloc[-1])

Iteration:      1,   Func. Count:      6,   Neg. LLF: 24533531.4143253
Iteration:      2,   Func. Count:     16,   Neg. LLF: -370.8457761164025
Optimization terminated successfully    (Exit mode 0)
            Current function value: -370.84577620011356
            Iterations: 6
            Function evaluations: 16
            Gradient evaluations: 2
-7.088914745481049e-05
0.012668075873532927


/home/codespace/.python/current/lib/python3.12/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001635. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


In [134]:
#Media pérdidas

media_perdidas_caso3 = -V_t_M*M_media_caso3 -V_t_N*N_media_caso3
print(media_perdidas_caso3)

48.547723884730715


In [137]:
factor = math.sqrt(M_std_caso3.iloc[-1] * N_std_caso3.iloc[-1])

# Calcular la desviación estándar de pérdidas
std_perdidas_caso3 = math.sqrt(
    (V_t_M**2) * (M_std_caso3.iloc[-1]**2) +
    (V_t_N**2) * (N_std_caso3.iloc[-1]**2) +
    2 * correlacion * V_t_M * V_t_N * factor
)

print(std_perdidas_caso2)

50229.03799967118


In [141]:
#VaR
VaR_caso3 = media_perdidas_caso3+ z*std_perdidas_caso3
print(VaR_caso3)

58607.41385416143


In [143]:
#ES 
ES_caso3 = media_perdidas_caso3 + std_perdidas_caso3 * (np.exp(-z**2 / 2) / (np.sqrt(2 * np.pi) * (1 - alpha)))
print(ES_caso3)

73483.72832649175
